## 0 - Imports

In [4]:
%load_ext autoreload
%autoreload

In [78]:
import scipy.io as spio
import pandas as pd
import numpy as np
from numpy import absolute as nabs
import matplotlib.pyplot as plt

## 1 - Load, clean, test data

In [1]:
def impmat(fname = 'M_processed.mat', writ = True):
    ''' import matlab crap, and turn it to pickles (or return panda df)'''
    mat = spio.loadmat(fname, squeeze_me=True)
    M = mat['M'] 
    head = ['time','ndc1','ndc2','ndc3',
            'Trade_Partner_Name', 'Distribution_Center_State','NDC','Distribution_Center_ID_(IC)',
    'Distribution_Center_Zip','Eff_Inv_(EU)','Eff_Inv_(PU)','Qty_Ord_(EU)',
            'Qty_Ord_(PU)']
    # get rid of ndc 1,2,3 because they're pieces of NCD
    # also get rid of purchase units, just use eatable
    # also get rid of states and zip code
    needed = [0,4,6,7,9,11]
    head_adj = [head[i] for i in needed]
    data = pd.DataFrame(M, columns=head)
    data["time"] = pd.to_datetime(data["time"], format='%Y%m%d', errors='coerce')

    if writ: # h5 allows your variable to be external
        dt = pd.HDFStore("drugdata.h5") # don't need to import/export! warning, though: huge
        dt['dat'] = data[head_adj] #
    return(data)


In [2]:
def test_hd5(p = 0, q = 0):
    """test data and run answers to intro quiz
    p is to print head of dataframe
    q prints quiz answers
    doesn't return anything
    mostly for access examples"""
    dt = pd.HDFStore("drugdata.h5")["dat"]

    header = dt.columns.tolist()
    # thanks @brock
    def q1(df):
        return(df.Trade_Partner_Name.unique())
    
    def q2(df):
        q2 = df.groupby('Trade_Partner_Name')['Distribution_Center_ID_(IC)'].nunique()
        q2max = q2.max()
        return(q2[q2 == q2max])
    
    def q3(df):
        q3df = df.loc[df["time"].dt.year == 2011] # can also use dt.month
        q3TotalSales = q3df.groupby('NDC')['Qty_Ord_(EU)'].sum()
        #print(q3TotalSales)
        q3sorted = q3TotalSales.sort_values(ascending = False).head()
        return(q3sorted)
    
    def q4(df):
        q4 = df['NDC'].value_counts()
        NDCLessThan60 = q4[q4 < 60]
        if (NDCLessThan60.size == 0):
            return(None)
        else:
            return(NDCLessThan60.size)
        
    def q5(df):
        q5 =  df.groupby('NDC')['Qty_Ord_(EU)'].std()
        q5max = q5.max()
        NDCHighestVariance = q5[q5 == q5max]
        return(NDCHighestVariance)
    
    def q6(df):
        q6 = df.groupby('NDC')['Qty_Ord_(EU)'].nunique()
        q6ZeroDemand = q6[q6 == 0]
        if (q6ZeroDemand.size == 0):
            return(None)
        else:
            return(q6ZeroDemand.size)
    
    if p:
        for col in header:
            print(dt[col].head())
    if q:
        answers = [q1(dt), q2(dt), q3(dt), q4(dt), q5(dt), q6(dt)]
        for i, ans in enumerate(answers):
            try:
                print('Question %d'%(i+1),  ans)
            except:
                print('Question %d'%(i+1) + str(ans))

In [7]:
#impmat(); # uncomment if never built h5 file
#test_hd5(q=1) # add p=1 or q = 1 to print stuff

In [8]:
def rem_neg_vals():
    ''' if you've just imported from the mat file,
    you need to run this to change the neg vals to 0 '''
    df = pd.HDFStore("drugdata.h5")["dat"]
    # set negative values to 0
    df.loc[df['Eff_Inv_(EU)'] < 0,'Eff_Inv_(EU)'] = 0
    df.loc[df['Qty_Ord_(EU)'] < 0,'Qty_Ord_(EU)'] = 0
    df.loc[df['Eff_Inv_(EU)'].isnull(), 'Eff_Inv_(EU)'] = 0
    df.loc[df['Qty_Ord_(EU)'].isnull(), 'Qty_Ord_(EU)'] = 0
    #print(df.head())
    return(True)

In [9]:
rem_neg_vals();

## 2 - Early queries

In [66]:
def top_selling(thr, p = 0):
    ''' in: minimum contributing percentage threshold
        if p, prints number and % of drugs above thr
        out: IDs of drugs above thr'''
    df = pd.HDFStore("drugdata.h5")["dat"]
    ind_total = df.groupby('NDC')['Qty_Ord_(EU)'].sum()
    sortsales = ind_total.sort_values(ascending = False)
    #print(sortsales)
    total = sum(ind_total.values)
    perc_total = 100 * sortsales / total
    clipped_above_total = perc_total[perc_total > thr]
    if p:
        print(len(clipped_above_total), sum(clipped_above_total.values))
    return(clipped_above_total.axes)

In [67]:
top_selling(1.5);

In [56]:
def weeks():
    ''' gives us a list of the weeks as a datetime Series '''
    df = pd.HDFStore("drugdata.h5")["dat"]

    return(pd.to_datetime(df.time.unique()).sort_values())

In [18]:
weeks();

In [138]:
def sales_exist():
    ''' want to check that every week has sales
        returns list of drug ids that have data for every year '''
    df = pd.HDFStore("drugdata.h5")["dat"]
    useless = {}
    years = [i for i in range(2007, 2018)]
    for drug in df.NDC.unique():
        useless[drug] = []
    for year in years:
        sales = df.loc[df.time.dt.year == year].groupby('NDC')['Qty_Ord_(EU)'].sum()
        for drug in df.NDC.unique():
            try:
                if sales[drug] == 0:
                    #print(drug, year) # have 0 sum
                    useless[drug].append(year)
            except:
                #print("broke by", drug, "in", year)
                useless[drug].append(year) # have NA or something?
    not_useless = []
    for did in useless.keys():
        if not useless[did]:
            not_useless.append(did)
    return(not_useless)

In [142]:
sales_exist();

## 3 - The meaty bits!

In [103]:
def sales(Year):
    """in: range of dates want studied 
    want to return the list of sales per date
    todo: break up by location if we want"""
    df = pd.HDFStore("drugdata.h5")["dat"]
    
    sel_drugs = [i for i in top_selling(1.5)[0]] # list of drug ids
    dates = df.loc[df.time.dt.year == Year] # choose only given year
    # gives DF of drugs by week; can change to 
    # ['NDC', 'time', DISTRO_id] if we want later
    window = dates.groupby(['NDC', 'time'])['Qty_Ord_(EU)'].sum()
    filt_window = window.loc[sel_drugs] # only want top drugs
    return(filt_window)

In [143]:
sales(2008);

In [23]:
def smape(f, d):
    ''' symmetric mean absolute percentage error
    in: vectors f = y_hat, d = y 
    out: the smape, yo '''
    n = len(f)
    num = np.sum(nabs(f - d))
    denom = np.sum(nabs(f) + nabs(d))
    return((1/n) * num/denom)

In [102]:
def frame_gen(year):
    ''' in: the year starting the frame
        out: 2 dfs, 3 years for training and 4th for test'''
    window = []
    for i in range(3):
        window.append(sales(year + i))
    # make a table of 3 years
    window = pd.concat(window)
    test_frame = sales(year + 3)
    return(window, test_frame)

In [30]:
def svr_try_0(tst, trn):
    ''' input: testing and training data 
        out: support vector regression score(?) '''
    
    from sklearn import preprocessing as pp
    X = df.loc[:, 2:].values
    y = df.loc[:, 1].values
    le = pp.LabelEncoder()
    #lb = pp.LabelBinerizer()
    y = le.fit_transform(y)
    
    from sklearn.cross_validation import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
    print(len(x_train))
    from sklearn.svm import SVR
    with open("out.txt", "w") as f:
        for i in range(-2, 2):
            svm = SVC(kernel="linear", C=10**i, random_state=0)
            svm.fit(x_train, y_train)
            f.write("c = " + str(10**i) + ", accuracy = " + 
            str(round(svm.score(x_test, y_test), 3)) + "\n")


In [26]:
def ARIMA(tst, trn):
    ''' Alright, Jack, do your thing. 
    I don't wanna touch this one. '''
    from statsmodel.tsa.arima_model import ARIMA
    pass

In [27]:
def NN_stuff(tst, trn):
    ''' All you, Collin '''
    pass

In [28]:
def lasso_stuff(tst, trn):
    ''' if we don't like those, we can try this too '''
    pass

## Plots

In [90]:
def svr_stuff(drug, tst, trn, lag_size):
    ''' need model for each drug id; maybe pass in main?
        test data, train data
        lag is number of previous vars '''
    from sklearn.svm import SVR
    print(drug)
    print(len(trn[drug]))
    vals = trn[drug].values
    from statsmodels.graphics.tsaplots import plot_acf
    plot_acf(vals, lags=lag_size)
    plt.show()
    #for i in range(1, lag_size+1):
        
    
    

In [94]:
def main(lag_size):
    dt = pd.HDFStore("drugdata.h5")["dat"]
    spread = [i for i in range(2007, 2015)] # you'll see
    dlist = top_selling(1.5)[0]
    thetaset = np.zeros(shape=(len(dlist), lag_size))# what do we want here???
    thetaset = pd.DataFrame(thetaset, index=dlist)
    for year in spread: # sliding window for analysis
        [trn, tst] = frame_gen(year)
        for drug in dlist:
            svr_stuff(drug,tst,trn,lag_size)
        return(0)
#         seth = svr_stuff(tst, trn)
#         collin = nn_stuff(tst, trn)
#         jack = ARIMA(tst, trn)
#         maybe = lasso_stuff(tst, trn)
#         which = [seth, collin, jack, maybe]
#         who = ["seth", "collin", "jack", "maybe"]
#         best_for_year = who[which.index(min(which))]
#         print("The best for", year, "is", best_for_year)
        
    

In [101]:
#main(10)
df = pd.HDFStore("drugdata.h5")["dat"]
[trn, tst] = frame_gen(2008)
dl = top_selling(1.5)[0]
for drug in dl:
    print(drug, trn[drug])

55.0 time
2008-01-06    3.205470e+06
2008-01-13    6.859200e+06
2008-01-20    5.527680e+06
2008-01-27    3.949440e+06
2008-02-03    4.913760e+06
2008-02-10    6.907413e+06
2008-02-17    6.006933e+06
2008-02-24    5.604693e+06
2008-03-02    4.251513e+06
2008-03-09    4.935573e+06
2008-03-16    4.974186e+06
2008-03-23    7.462506e+06
2008-03-30    6.333546e+06
2008-04-06    4.803306e+06
2008-04-13    5.072319e+06
2008-04-20    5.207679e+06
2008-04-27    4.737492e+06
2008-05-04    5.153386e+06
2008-05-11    4.670079e+06
2008-05-18    6.478779e+06
2008-05-25    5.787306e+06
2008-06-01    4.796586e+06
2008-06-08    5.830506e+06
2008-06-15    5.997546e+06
2008-06-22    4.773279e+06
2008-06-29    7.030506e+06
2008-07-06    4.841706e+06
2008-07-13    4.878186e+06
2008-07-20    4.400853e+06
2008-07-27    5.474373e+06
                  ...     
2010-06-06    3.663150e+06
2010-06-13    3.283590e+06
2010-06-20    3.657150e+06
2010-06-27    8.238930e+06
2010-07-04    3.356010e+06
2010-07-11    3.81

In [ ]:
correlations, seem reasonable: [0, 2, 0, 0, 2, 1, 0]